<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Deploy a Classification Pipeline with BYOM & ONNX in 5 minutes
 <br>       
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 150px; height: auto; margin-top: 20pt;">
  <br>
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>
   In this demo, we will demonstrate how to train and deploy a classification Pipeline using sklearn Preprocessing and Classification Algorithm through conversion to ONNX and deployment in-DB with Bring Your Own Model.
       </p>

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial;'><b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import getpass

import teradataml as tdml
try:
    tdml.display.enable_ui = False # just display the normal DataFrame, available from version 20.00.00.04
except:
    pass

<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = tdml.create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
tdml.execute_sql('''SET query_band='DEMO=PP_Recipe_BYOM.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>The focus is in this notebook is a recipe, i.e. a generic approach. Hence we will use the famous Titanic dataset.</p>   


In [ ]:
from sklearn.datasets import fetch_openml
df = fetch_openml('titanic', version=1, as_frame=True)['frame']

tdml.copy_to_sql(
    df.reset_index().rename(columns={"index": "row_id"})[
        [
            'row_id',
            'survived',
            'sex',
            'embarked',
            'pclass',
            'age',
            'sibsp',
            'parch',
            'fare'
        ]
    ],
    "titanic_input",
    if_exists="replace",
    primary_index="row_id"
)

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>3. Model Pipeline Fitting </b></p>

<p style = 'font-size:16px;font-family:Arial'>We will use SimpleImputer for numeric variables, OneHotEncoder for categorical variables, and a RandomForestClassifier as the actual model.</p>

In [ ]:
DF_Titanic = tdml.DataFrame("titanic_input")
DF_Titanic

In [ ]:
DF_Titanic.tdtypes

In [ ]:
df_titanic = DF_Titanic.to_pandas()
df_titanic.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Drop rows with missing target
df = df_titanic.dropna(subset=['survived'])
X = df.drop(columns='survived')
y = df['survived']

# Select features
cat_features = ['sex', 'embarked']
num_features = ['age', 'sibsp', 'parch', 'fare', 'pclass']

# Preprocessing
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), num_features),
    ('encoder', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

# Pipeline
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=0))
])

pipeline.fit(X[cat_features + num_features], y)


<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>4. Pipeline Conversion to ONNX </b></p>

        

In [ ]:
DF_Titanic.tdtypes

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType

initial_type = [
    ('sex', StringTensorType([None, 1])),
    ('embarked', StringTensorType([None, 1])),
    ('pclass', FloatTensorType([None, 1])),
    ('age', FloatTensorType([None, 1])),
    ('sibsp', FloatTensorType([None, 1])),
    ('parch', FloatTensorType([None, 1])),
    ('fare', FloatTensorType([None, 1]))
]

# Convert and save
onnx_model = convert_sklearn(pipeline, initial_types=initial_type)
with open("titanic_pipeline.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>5. Inference with Bring Your Own Model (BYOM) function ONNXPredict </b></p>


In [ ]:
tdml.configure.byom_install_location = "mldb"

In [ ]:
model_table_name = "titanic_model_table"
model_id = "rf_pipeline_titanic"

In [ ]:
try:
    tdml.db_drop_table(model_table_name)
except:
    pass

In [ ]:
tdml.save_byom(model_id, "titanic_pipeline.onnx", table_name = model_table_name)

In [ ]:
tdml.DataFrame(model_table_name)

In [ ]:
onnx_predict_obj = tdml.ONNXPredict(
    newdata=DF_Titanic,
    modeldata=tdml.DataFrame(model_table_name),
    accumulate=["row_id", "survived"],
    overwrite_cached_models="TRUE"
)

In [ ]:
onnx_predict_obj.result

In [ ]:
onnx_query = onnx_predict_obj.show_query()
print(onnx_query)

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>6. Deploy Inference as View </b></p>

<p style = 'font-size:16px;font-family:Arial'>
This allows us to integrate the Inference just as another normal ETL Job.</p>

In [ ]:
tdml.execute_sql(f"""
REPLACE VIEW titanic_inference_v AS
{onnx_query}
""")

In [ ]:
tdml.DataFrame("titanic_inference_v")

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Cleanup</b></p>

In [ ]:
try:
    tdml.db_drop_table("titanic_input")
except:
     pass

In [ ]:
try:
    tdml.db_drop_table("titanic_model_table")
except:
     pass

In [ ]:
try:
    tdml.db_drop_view("titanic_inference_v")
except:
     pass

In [ ]:
tdml.remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Dataset</b><br>We have used the Titanic dataset, available via the sklearn.datasets module. This dataset contains information on 891 passengers who were aboard the Titanic, with the goal of predicting survival outcomes. It includes a mix of numerical and categorical features such as age, sex, passenger class, number of siblings/spouses aboard, number of parents/children aboard, fare paid, and port of embarkation. The target variable indicates whether a passenger survived or not.</p>


<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>